# Automating Rhino and Revit together using Python Script and Hops
#### Writen by: Aman, Jayedi
#### iLab, University of Missouri
#### 4th February, 2023

## Pandas

Pandas is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool, built on top of the Python programming language.
Pandas is built on top of the [NumPy](https://numpy.org/) package, meaning a lot of the structure of NumPy is used or replicated in Pandas. Data in pandas is often used to feed statistical analysis in [SciPy](https://www.scipy.org/), plotting functions from [Matplotlib](https://matplotlib.org/), and machine learning algorithms in [Scikit-learn](https://scikit-learn.org/stable/).

### Installation

You can install libraries using [Pip](https://pypi.org/project/pip/) which is a package installer for Python. You can also create a virtual environment for your installations using [Anaconda](https://www.anaconda.com/products/individual#Downloads)  [recommended]. 

[pandas installation link](https://pandas.pydata.org/getting_started.html)

In [78]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings('ignore')

import numpy as np

# general pyhthon libs import
import base64
import json
import requests

#For iteration process
import sys
from os import listdir
from os.path import isfile, join
import pandas as pd
import os
import time
import csv

#### Set Directory

In [79]:

# Python program to explain os.getcwd() method 
          
# importing os module 
import os 
      
# Get the current working 
# directory (CWD) 
cwd = os.getcwd() 
      
# Print the current working 
# directory (CWD) 
print("Current working directory:", cwd)

Current working directory: H:\MIZZOU\_MotherFolderAmanWorks\Research With Dr Jong Bum\_2022-2023_AT&T_DigitalTwin\_WorkUpdates_DigitalTwinResearch_AT&T\_Spring23Works\20230205_Final_Iteration&AutomationUsingHops


### Rhino 3dm

[Rhino3dm](https://pypi.org/project/rhino3dm/) is a set of libraries based on the OpenNURBS geometry library with a "RhinoCommon" style. This provides the ability to access and manipulate geometry through .NET, Python or JavaScript applications independent of Rhino.

[Rhino3dm Link](https://github.com/mcneel/rhino3dm)

`pip install rhino3dm`

[Rhino Compute Link](https://pypi.org/project/compute-rhino3d/)

`pip install compute-rhino3d`

In [80]:
# rhino 3dm import
# from rhino3dm import *
import rhino3dm as rhino3dm
import compute_rhino3d.Util

## Grasshopper + HOPS

1. Open Rhino 7
2. type `PackageManger` in the Rhino command line.
3. search for *Hops*
4. Select *Hops* and install
5. Restart Rhino

[Hops Website Link](https://developer.rhino3d.com/guides/grasshopper/hops-component/)

In [81]:
# Rhino compute by default launches on port 6500
compute_url = "http://localhost:8081/"

# set the URL
compute_rhino3d.Util.url = compute_url
# no auth token required
compute_rhino3d.Util.authToken = ""

# test, should return version object
version_test = requests.get(compute_url + '/version')
json.loads(version_test.content)

{'rhino': '7.24.22308.15001', 'compute': '1.0.0.0', 'git_sha': None}

#### Load Grasshopper File

In [82]:
'''
Load Grasshopper model via File3dm.Read
'''

rhino_dir = "../Rhino/"
#gh_path = "20220915_DigitalTwinForHops.ghx"
gh_path = "20230205_DigitalTwinForHops.ghx"
file_path = rhino_dir + gh_path
print(file_path)
# read the 3dm file
ghFile = rhino3dm.File3dm.Read(file_path)


gh_geo = open("Rhino/20230205_DigitalTwinForHops.ghx", mode="r", encoding="utf-8-sig").read()
gh_geo_bytes = gh_geo.encode("utf-8")
gh_geo_encoded = base64.b64encode(gh_geo_bytes)
gh_geo_decoded = gh_geo_encoded.decode("utf-8")

../Rhino/20230205_DigitalTwinForHops.ghx


### Get the data from sensors

In [83]:
# # Inputs: We'll get the inputs from the sensors

# # adjust between 0 and 0.9
# id_sim = 500

# # # adjust between 1 and 50
# # Equipment = 5

# # #Lighting
# # #Light = #multiply lux to 0.0079

# # # adjust between 0 and 1
# # MOCarbonFactor = 0.69

# # # ---------------------

In [84]:
# # payload
# geo_payload = {
#     "algo": gh_geo_decoded,
#     "pointer": None,
#     "values": [
#         {
#             "ParamName": "id_sim",
#             "InnerTree": {
#                 "{ 0; }": [
#                     {
#                         "type": "System.Double",
#                         "data": id_sim
#                     }
#                 ]
#             }
#         }
#     ]
# }

# # send HTTP request to Rhino Compute Server
# res = requests.post(compute_url + "grasshopper", json=geo_payload)

# # print("status code: {}".format(res.status_code))

# # deserialize response obj293.29ect
# response_object = json.loads(res.content)['values']

# # geometry_output = [result for result in response_object if result['ParamName'] == 'RH_OUT:geometry'][0]['InnerTree']['{0;0;0;0;0}']
# # print("number of buildings generated: {}".format(len(geometry_output)))

# OperationalCarbon = [result for result in response_object if result['ParamName'] == 'RH_OUT:emissions_carbon'][0]['InnerTree']['{0}'][0]['data']
# print('Operational Carbon: {}'.format(OperationalCarbon))

# # EUI = [result for result in response_object if result['ParamName'] == 'RH_OUT:EUI'][0]['InnerTree']['{0}'][0]['data']
# # print('Energy Use Intensity: {}'.format(EUI))

In [85]:
# Initialize the loop variable
id_sim = 0

# Loop until the loop variable reaches 744
while id_sim < 744:
    
    # payload
    geo_payload = {
        "algo": gh_geo_decoded,
        "pointer": None,
        "values": [
            {
                "ParamName": "id_sim",
                "InnerTree": {
                    "{ 0; }": [
                        {
                            "type": "System.Double",
                            "data": id_sim
                        }
                    ]
                }
            }
        ]
    }

    # send HTTP request to Rhino Compute Server
    res = requests.post(compute_url + "grasshopper", json=geo_payload)

    # print("status code: {}".format(res.status_code))

    # deserialize response obj293.29ect
    response_object = json.loads(res.content)['values']


    emissions_carbon = [result for result in response_object if result['ParamName'] == 'RH_OUT:emissions_carbon'][0]['InnerTree']['{0}'][0]['data']
    
    print('id- {}'.format(id_sim)+ ', ' + 'Operational Carbon: {}'.format(emissions_carbon))
    
    
        # Write the result to a CSV file
    with open('emissions_carbon_results.csv', 'a') as file:
        writer = csv.writer(file)
        writer.writerow([id_sim, emissions_carbon])

    # Increment the loop variable
    id_sim += 1

    # Wait for 1 second before moving to the next iteration
    time.sleep(0)




id- 0, Operational Carbon: 1.6060818082191779
id- 1, Operational Carbon: 1.2418317534246575
id- 2, Operational Carbon: 1.1361984246575341
id- 3, Operational Carbon: 1.1361984246575341
id- 4, Operational Carbon: 1.1361984246575341
id- 5, Operational Carbon: 1.1361984246575341
id- 6, Operational Carbon: 1.1361984246575341
id- 7, Operational Carbon: 1.279813417808219
id- 8, Operational Carbon: 1.2418317534246575
id- 9, Operational Carbon: 1.1361984246575341
id- 10, Operational Carbon: 1.1361984246575341
id- 11, Operational Carbon: 1.1361984246575341
id- 12, Operational Carbon: 1.1361984246575341
id- 13, Operational Carbon: 1.1361984246575341
id- 14, Operational Carbon: 1.1361984246575341
id- 15, Operational Carbon: 1.1361984246575341
id- 16, Operational Carbon: 1.2418317534246575
id- 17, Operational Carbon: 1.2134623561643836
id- 18, Operational Carbon: 1.279813417808219
id- 19, Operational Carbon: 1.6060818082191779
id- 20, Operational Carbon: 1.8134263356164382
id- 21, Operational Carbo

In [88]:
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot, row, column
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource, DataTable, TableColumn

# Read the CSV file into a pandas DataFrame
data = pd.read_csv('emissions_carbon_results.csv', names=['ID', 'Emissions Carbon'])
source = ColumnDataSource(data)

# Create a line plot
line_plot = figure(title="Line Plot", x_axis_label="ID", y_axis_label="Emissions Carbon")
line_plot.line(x='ID', y='Emissions Carbon', source=source, line_width=2)

# Create a bar plot
bar_plot = figure(title="Bar Plot", x_axis_label="ID", y_axis_label="Emissions Carbon")
bar_plot.vbar(x='ID', top='Emissions Carbon', width=0.9, source=source)

# Create a scatter plot
scatter_plot = figure(title="Scatter Plot", x_axis_label="ID", y_axis_label="Emissions Carbon")
scatter_plot.scatter(x='ID', y='Emissions Carbon', size=10, color="red", source=source)

# Create a heading for the dashboard
heading = figure(title="AT&T Digital Twin", plot_width=600, plot_height=50)

# Create a data table
columns = [
    TableColumn(field="ID", title="ID"),
    TableColumn(field="Emissions Carbon", title="Emissions Carbon")
]
data_table = DataTable(source=source, columns=columns, width=600, height=200)

# Organize the plots and heading into a grid layout
layout = gridplot([[line_plot, bar_plot], [scatter_plot, data_table]])
final_layout = column(row(heading), layout)

# Save the layout to an HTML file
output_file("AT&T Digital Twin Dashboard.html")
show(final_layout)


In [87]:
data

,ID,Emissions Carbon
0,0,1.606082
1,1,1.241832
2,2,1.136198
3,3,1.136198
4,4,1.136198
5,5,1.136198
6,6,1.136198
7,7,1.279813
8,8,1.241832
9,9,1.136198
